In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import math
from itertools import combinations, product

In this notebook, we will use the Shortest Path formulation to get out optimal tours. 

In [2]:
# Function will convert our datfile of x,y coordinates into dataframe: vertices
def TSP_vertices(file_name):
    data  = open(file_name, 'r')
    lines = data.readlines()[1:]
    x = []
    y = []
    for line in lines:
        p = line.split()
        x.append(float(p[0]))
        y.append(float(p[1]))
    data.close()
    # Transform into dataframe
    vertices = pd.DataFrame({'x':x, 'y':y})
    
    return vertices

In [3]:
# Will convert Euclidean distance for our coordinates as cost components for model
def euclideanDistMatrix(TSP):
    euclidean_cost = []
    for i in range(len(TSP)):
        row_distances = []
        for j in range(len(TSP)):
            row_distances.append(math.sqrt( (TSP['x'][i]-TSP['x'][j])**2 + (TSP['y'][i]-TSP['y'][j])**2 ))
        euclidean_cost.append(row_distances)
    return euclidean_cost  

In [4]:
### FUNCTION. This function will take the .dat file of x,y coordinates and do the following. It will create the cost
### matrix for each pair of vertices. We will then implement a shortest path formulation for the TSP data. It will 
### output the tour and its associated objective cost to the user. 

def shortestPath(file_name):
    ### DATA Collection
    
    # Obtain our cost for each arc from distance matrix
    verticesDF = TSP_vertices(file_name)
    cost = euclideanDistMatrix(verticesDF)

    # Create a list of the nodes: vertices
    V = []
    for i in range(len(verticesDF)):
        V.append(i)
    
    # Create a list A of all tuples of (i,j,t) which represent arc from i to j at layer t
    A = [(i, j, t) for i in V for j in V for t in V if i != j]
    
    # Create a dictionary that has each (i,j,t) tuple with its cost of going from i to j for layer t; c
    # Note: Going from a fixed i to fixed j has same cost regardless of layer t. 
    # Note: the (i,j,t) tuple key in the c dictionary serves as our binary decision variable
    c = {(i,j,t): cost[i][j] for i, j, t in product(V, V, V) if i != j} 
       
    ### MODEL FORMULATION for the Shortest Path with Constraints 

    # Instantiate Model
    model = Model('Shortest Path')
    model.ModelSense = GRB.MINIMIZE

    # Instantiate the variables and objective function
    x = model.addVars(A, vtype=GRB.BINARY, name='e')
    # x = model.addVars(A, lb = 0, ub = 1, vtype=GRB.INTEGER, name='e')
    model.modelSense = GRB.MINIMIZE
    model.setObjective(quicksum(x[i, j, t]*c[i, j, t] for i, j, t in A))  # objective function
    
    # first constraint 
    model.addConstrs(quicksum(x[0,j,0] for j in V[1:] if j != 0) == 1 for j in V[1:] if j != 0) 
    # second constraint: first layer 
    model.addConstrs(quicksum(x[i,j,1] for j in V[1:] if i != j) - x[0,i,0] == 0 for i in V[1:])
    # third constraint: t'th layer
    model.addConstrs(quicksum(x[i,j,t+1] for j in V[1:] if j != i) - quicksum(x[j,i,t] for j in V[1:] if j != i) == 0 for i in V[1:] for t in V[1:len(V)-2])
    # fourth constraint: n - 1 layer
    model.addConstrs(x[i,0,len(V)-1] - quicksum(x[j,i,len(V)-2] for j in V[1:] if j != i) == 0 for i in V[1:] )
    # fifth constaint: last layer
    model.addConstrs(quicksum(x[i,0,len(V)-1] for i in V[1:]) == 1 for i in V[1:])
    # sixth constraint: not repeating the same vertex
    # model.addConstrs(quicksum(x[i,j,t] for t in V[1:len(V)-2] for j in V[1:] if j != i) + x[i,0,len(V)-1] <= 1 for i in V[1:])
    model.addConstrs(quicksum(quicksum(x[i,j,t] for j in V[1:] if j != i) for t in V[1:len(V)-1]) + x[i,0,len(V)-1] <= 1 for i in V[1:])
    
    # Solve the model and print runtime. 
    model.Params.TimeLimit = 600
    model.optimize()
    print('The runtime was', model.Runtime, 'seconds.')
       
    ### UNCOMMENT LINE BELOW if you want to save a file of the formulation. Open with Microsoft Word.
    # model.write('Shortest Path.lp')
    
    ### UNCOMMENT BELOW IF you want the tour arcs outputted.
    #for v in model.getVars():
     #   if v.x > 0.5:
      #      print(v)
    
    return model

In [7]:
### This is a Linear Relaxation of the Shortest Path Formulation 
def shortestPath_LR(file_name):
    ### DATA Collection
    
    # Obtain our cost for each arc from distance matrix
    verticesDF = TSP_vertices(file_name)
    cost = euclideanDistMatrix(verticesDF)

    # Create a list of the nodes: vertices
    V = []
    for i in range(len(verticesDF)):
        V.append(i)
    
    # Create a list A of all tuples of (i,j,t) which represent arc from i to j at layer t
    A = [(i, j, t) for i in V for j in V for t in V if i != j]
    
    # Create a dictionary that has each (i,j,t) tuple with its cost of going from i to j for layer t; c
    # Note: Going from a fixed i to fixed j has same cost regardless of layer t. 
    # Note: the (i,j,t) tuple key in the c dictionary serves as our binary decision variable
    c = {(i,j,t): cost[i][j] for i, j, t in product(V, V, V) if i != j} 
    
    
    ### MODEL FORMULATION for the Shortest Path with Constraints 

    # Instantiate Model
    model = Model('Shortest Path')
    model.ModelSense = GRB.MINIMIZE

    # Instantiate the variables and objective function
    x = model.addVars(A, lb=0, ub = 1, vtype=GRB.CONTINUOUS, name='e')
    
    # Set the objective function
    model.modelSense = GRB.MINIMIZE
    model.setObjective(quicksum(x[i, j, t]*c[i, j, t] for i, j, t in A))  # objective function
    
    # first constraint 
    model.addConstrs(quicksum(x[0,j,0] for j in V[1:] if j != 0) == 1 for j in V[1:] if j != 0) 
    # second constraint: first layer 
    model.addConstrs(quicksum(x[i,j,1] for j in V[1:] if i != j) - x[0,i,0] == 0 for i in V[1:])
    # third constraint: t'th layer
    model.addConstrs(quicksum(x[i,j,t+1] for j in V[1:] if j != i) - quicksum(x[j,i,t] for j in V[1:] if j != i) == 0 for i in V[1:] for t in V[1:len(V)-2])
    # fourth constraint: n - 1 layer
    model.addConstrs(x[i,0,len(V)-1] - quicksum(x[j,i,len(V)-2] for j in V[1:] if j != i) == 0 for i in V[1:] )
    # fifth constaint: last layer
    model.addConstrs(quicksum(x[i,0,len(V)-1] for i in V[1:]) == 1 for i in V[1:])
    # sixth constraint: not repeating the same vertex
    # model.addConstrs(quicksum(x[i,j,t] for t in V[1:len(V)-2] for j in V[1:] if j != i) + x[i,0,len(V)-1] <= 1 for i in V[1:])
    model.addConstrs(quicksum(quicksum(x[i,j,t] for j in V[1:] if j != i) for t in V[1:len(V)-1]) + x[i,0,len(V)-1] <= 1 for i in V[1:])
    # Solve the model. 
    model.Params.TimeLimit = 600
    model.optimize()
    print('The runtime was', model.Runtime, 'seconds.')
    
    ### UNCOMMENT LINE BELOW if you want to save a file of the formulation. Open with Microsoft Word.
    # model.write('Shortest Path.lp')
    
    ### UNCOMMENT BELOW IF you want the tour arcs outputted.
    #for v in model.getVars():
     #   if v.x > 0.5:
      #      print(v)
    
    return model

The following are the Shortest Path formulation integer and Linear Relaxation solutions for two instances of size 15.

In [8]:
shortestPath('TSP_instance_n_15_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 238 rows, 3150 columns and 7532 nonzeros
Model fingerprint: 0x4fa8d09b
Variable types: 0 continuous, 3150 integer (3150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 145.1984008
Presolve removed 34 rows and 764 columns
Presolve time: 0.06s
Presolved: 204 rows, 2386 columns, 7152 nonzeros
Variable types: 0 continuous, 2386 integer (2386 binary)

Root relaxation: objective 5.855349e+01, 710 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   58.55349    0   81  145.19840   58.55349  59.7%     -    0s
H    0     0                

<gurobi.Model MIP instance Shortest Path: 238 constrs, 3150 vars, Parameter changes: TimeLimit=600.0>

In [7]:
shortestPath_LR('TSP_instance_n_15_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 238 rows, 3150 columns and 7532 nonzeros
Model fingerprint: 0x05121d73
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 46 rows and 775 columns
Presolve time: 0.03s
Presolved: 192 rows, 2375 columns, 7116 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9527910e+00   1.000000e+00   0.000000e+00      0s
     645    5.8553491e+01   0.000000e+00   0.000000e+00      0s

Solved in 645 iterations and 0.07 seconds
Optimal objective  5.855349134e+01
The runtime was 0.07555508613586426 seconds.


<gurobi.Model Continuous instance Shortest Path: 238 constrs, 3150 vars, Parameter changes: TimeLimit=600.0>

In [8]:
shortestPath('TSP_instance_n_15_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 238 rows, 3150 columns and 7532 nonzeros
Model fingerprint: 0xdb47137d
Variable types: 0 continuous, 3150 integer (3150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 178.9752490
Presolve removed 34 rows and 764 columns
Presolve time: 0.05s
Presolved: 204 rows, 2386 columns, 7152 nonzeros
Variable types: 0 continuous, 2386 integer (2386 binary)

Root relaxation: objective 6.405054e+01, 773 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   64.05054    0   75  178.97525   64.05054  64.2%     -    0s
H    0     0                

<gurobi.Model MIP instance Shortest Path: 238 constrs, 3150 vars, Parameter changes: TimeLimit=600.0>

In [9]:
shortestPath_LR('TSP_instance_n_15_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 238 rows, 3150 columns and 7532 nonzeros
Model fingerprint: 0x508c53a2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 46 rows and 775 columns
Presolve time: 0.02s
Presolved: 192 rows, 2375 columns, 7116 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.6841947e+00   1.000000e+00   0.000000e+00      0s
     580    6.4050539e+01   0.000000e+00   0.000000e+00      0s

Solved in 580 iterations and 0.07 seconds
Optimal objective  6.405053888e+01
The runtime was 0.07703685760498047 seconds.


<gurobi.Model Continuous instance Shortest Path: 238 constrs, 3150 vars, Parameter changes: TimeLimit=600.0>

The following are the Shortest Path formulation integer and Linear Relaxation solutions for two instances of size 20.

In [13]:
shortestPath('TSP_instance_n_20_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 418 rows, 7600 columns and 19247 nonzeros
Model fingerprint: 0xfd0da557
Variable types: 0 continuous, 7600 integer (7600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 172.8819495
Presolve removed 41 rows and 1411 columns
Presolve time: 0.08s
Presolved: 377 rows, 6189 columns, 18553 nonzeros
Variable types: 0 continuous, 6189 integer (6189 binary)

Root relaxation: objective 5.649531e+01, 1589 iterations, 0.12 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   56.49531    0  228  172.88195   56.49531  67.3%     -    0s
H    0     0            

<gurobi.Model MIP instance Shortest Path: 418 constrs, 7600 vars, Parameter changes: TimeLimit=600.0>

In [11]:
shortestPath_LR('TSP_instance_n_20_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 418 rows, 7600 columns and 19247 nonzeros
Model fingerprint: 0x17bda1fc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 42 rows and 1411 columns
Presolve time: 0.04s
Presolved: 376 rows, 6189 columns, 18517 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3391107e+00   2.000000e+00   0.000000e+00      0s
    1305    5.6495307e+01   0.000000e+00   0.000000e+00      0s

Solved in 1305 iterations and 0.16 seconds
Optimal objective  5.649530695e+01
The runtime was 0.16519808769226074 seconds.


<gurobi.Model Continuous instance Shortest Path: 418 constrs, 7600 vars, Parameter changes: TimeLimit=600.0>

In [12]:
shortestPath('TSP_instance_n_20_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 418 rows, 7600 columns and 19247 nonzeros
Model fingerprint: 0x50d319e5
Variable types: 0 continuous, 7600 integer (7600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 207.1145823
Presolve removed 41 rows and 1411 columns
Presolve time: 0.08s
Presolved: 377 rows, 6189 columns, 18553 nonzeros
Variable types: 0 continuous, 6189 integer (6189 binary)

Root relaxation: objective 6.981832e+01, 1612 iterations, 0.14 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   69.81832    0  166  207.11458   69.81832  66.3%     -    0s
H    0     0            

<gurobi.Model MIP instance Shortest Path: 418 constrs, 7600 vars, Parameter changes: TimeLimit=600.0>

In [24]:
shortestPath_LR('TSP_instance_n_20_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 418 rows, 7600 columns and 19247 nonzeros
Model fingerprint: 0x0ce38f08
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 42 rows and 1411 columns
Presolve time: 0.05s
Presolved: 376 rows, 6189 columns, 18517 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.8698322e+00   2.000000e+00   0.000000e+00      0s
    1655    6.9818322e+01   0.000000e+00   0.000000e+00      0s

Solved in 1655 iterations and 0.20 seconds
Optimal objective  6.981832207e+01
The runtime was 0.1994619369506836 seconds.


<gurobi.Model Continuous instance Shortest Path: 418 constrs, 7600 vars, Parameter changes: TimeLimit=600.0>

The following are the Shortest Path formulation integer and Linear Relaxation solutions for two instances of size 25.

In [14]:
shortestPath('TSP_instance_n_25_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 648 rows, 15000 columns and 39312 nonzeros
Model fingerprint: 0xe90ae561
Variable types: 0 continuous, 15000 integer (15000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 235.8197173
Presolve removed 46 rows and 2256 columns
Presolve time: 0.11s
Presolved: 602 rows, 12744 columns, 37680 nonzeros
Variable types: 0 continuous, 12744 integer (12744 binary)

Root relaxation: objective 6.419705e+01, 2876 iterations, 0.35 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   64.19705    0  249  235.81972   64.19705  72.8%     -    0s
H    0     0      

<gurobi.Model MIP instance Shortest Path: 648 constrs, 15000 vars, Parameter changes: TimeLimit=600.0>

In [15]:
shortestPath_LR('TSP_instance_n_25_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 648 rows, 15000 columns and 39312 nonzeros
Model fingerprint: 0x55516389
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 52 rows and 2261 columns
Presolve time: 0.12s
Presolved: 596 rows, 12739 columns, 38152 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.587e+04
 Factor NZ  : 3.392e+04 (roughly 6 MBytes of memory)
 Factor Ops : 2.031e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.17204232e+04  2.24081118e+02  5.30e+01 3.29e+00  1.89e-01     0s
   1   

<gurobi.Model Continuous instance Shortest Path_copy: 648 constrs, 15000 vars, Parameter changes: TimeLimit=600.0>

In [16]:
shortestPath('TSP_instance_n_25_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 648 rows, 15000 columns and 39312 nonzeros
Model fingerprint: 0xb4bd8fa8
Variable types: 0 continuous, 15000 integer (15000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 270.9638924
Presolve removed 46 rows and 2256 columns
Presolve time: 0.11s
Presolved: 602 rows, 12744 columns, 37680 nonzeros
Variable types: 0 continuous, 12744 integer (12744 binary)

Root relaxation: objective 7.143936e+01, 2782 iterations, 0.36 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   71.43936    0  224  270.96389   71.43936  73.6%     -    0s
H    0     0      

<gurobi.Model MIP instance Shortest Path: 648 constrs, 15000 vars, Parameter changes: TimeLimit=600.0>

In [17]:
shortestPath_LR('TSP_instance_n_25_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 648 rows, 15000 columns and 39312 nonzeros
Model fingerprint: 0x6d0d7f78
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 52 rows and 2261 columns
Presolve time: 0.08s
Presolved: 596 rows, 12739 columns, 38152 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.587e+04
 Factor NZ  : 3.392e+04 (roughly 6 MBytes of memory)
 Factor Ops : 2.031e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.48485425e+04  2.50776024e+00  5.30e+01 9.96e-02  7.58e-01     0s
   1   

<gurobi.Model Continuous instance Shortest Path_copy: 648 constrs, 15000 vars, Parameter changes: TimeLimit=600.0>

The following are the Shortest Path formulation integer and Linear Relaxation solutions for two instances of size 30.

In [18]:
shortestPath('TSP_instance_n_30_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 928 rows, 26100 columns and 69977 nonzeros
Model fingerprint: 0x6b09c6e3
Variable types: 0 continuous, 26100 integer (26100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 272.8647178
Presolve removed 56 rows and 3306 columns
Presolve time: 0.17s
Presolved: 872 rows, 22794 columns, 67570 nonzeros
Variable types: 0 continuous, 22794 integer (22794 binary)

Root relaxation: objective 7.166540e+01, 5149 iterations, 1.23 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   71.66540    0  495  272.86472   71.66540  73.7%     -    1s
H    0     0      

  3492  1977   85.44688   32  352   89.62315   83.03856  7.35%   265  347s
  3578  2015   87.03478   63  218   89.62315   83.04714  7.34%   269  353s
  3710  2111   87.44308   40  403   89.62315   83.05414  7.33%   271  359s
  3939  2175   89.30864   43  394   89.62315   83.12123  7.25%   267  365s
  4077  2290     cutoff   49        89.62315   83.17957  7.19%   267  372s
  4252  2387   88.44951   38  243   89.62315   83.23098  7.13%   265  379s
  4417  2553     cutoff   48        89.62315   83.23524  7.13%   264  386s
  4624  2745   89.34504   57  341   89.62315   83.25768  7.10%   261  394s
  4879  2851   89.00174   57  268   89.62315   83.32902  7.02%   258  401s
  5071  2986   86.45715   47  431   89.62315   83.38119  6.96%   258  409s
  5304  3156   85.67532   41  464   89.62315   83.41076  6.93%   258  417s
  5512  3309   87.08873   53  375   89.62315   83.45530  6.88%   258  426s
  5752  3427   87.07797   22  309   89.62315   83.51366  6.82%   257  436s
  5975  3463     cutoff  

<gurobi.Model MIP instance Shortest Path: 928 constrs, 26100 vars, Parameter changes: TimeLimit=600.0>

In [19]:
shortestPath_LR('TSP_instance_n_30_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 928 rows, 26100 columns and 69977 nonzeros
Model fingerprint: 0xead8ceea
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 62 rows and 3311 columns
Presolve time: 0.10s
Presolved: 866 rows, 22789 columns, 68287 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.620e+04
 Factor NZ  : 5.994e+04 (roughly 10 MBytes of memory)
 Factor Ops : 4.345e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.08586333e+04  1.32500461e+01  7.85e+01 1.28e+01  1.07e+00     0s
   1  

<gurobi.Model Continuous instance Shortest Path_copy: 928 constrs, 26100 vars, Parameter changes: TimeLimit=600.0>

In [20]:
shortestPath('TSP_instance_n_30_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 928 rows, 26100 columns and 69977 nonzeros
Model fingerprint: 0xf521fda2
Variable types: 0 continuous, 26100 integer (26100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 341.8519896
Presolve removed 56 rows and 3306 columns
Presolve time: 0.43s
Presolved: 872 rows, 22794 columns, 67570 nonzeros
Variable types: 0 continuous, 22794 integer (22794 binary)

Root relaxation: objective 6.865328e+01, 3324 iterations, 1.47 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   68.65328    0  408  341.85199   68.65328  79.9%     -    2s
H    0     0      

<gurobi.Model MIP instance Shortest Path: 928 constrs, 26100 vars, Parameter changes: TimeLimit=600.0>

In [21]:
shortestPath_LR('TSP_instance_n_30_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 928 rows, 26100 columns and 69977 nonzeros
Model fingerprint: 0xd02fad9d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 62 rows and 3311 columns
Presolve time: 0.09s
Presolved: 866 rows, 22789 columns, 68287 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.620e+04
 Factor NZ  : 5.994e+04 (roughly 10 MBytes of memory)
 Factor Ops : 4.345e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.60416617e+04  2.50776024e+00  7.85e+01 9.96e-02  7.32e-01     0s
   1  

<gurobi.Model Continuous instance Shortest Path_copy: 928 constrs, 26100 vars, Parameter changes: TimeLimit=600.0>

The following are the Shortest Path formulation integer and Linear Relaxation solutions for two instances of size 50.

In [22]:
shortestPath('TSP_instance_n_50_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 2548 rows, 122500 columns and 343637 nonzeros
Model fingerprint: 0x3b515084
Variable types: 0 continuous, 122500 integer (122500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 465.2221234
Presolve removed 96 rows and 9506 columns
Presolve time: 0.98s
Presolved: 2452 rows, 112994 columns, 336630 nonzeros
Variable types: 0 continuous, 112994 integer (112994 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    5163    4.9016140e+01   3.929692e+01   0.000000e+00      5s
    9272    6.2096380e+01   5.734203e+02   0.000000e+00     10s
   13205    7.3144082e+01   1.105868e+03   0.000000e+00     15s
   17123    8.6570797e+01   1

<gurobi.Model MIP instance Shortest Path: 2548 constrs, 122500 vars, Parameter changes: TimeLimit=600.0>

In [23]:
shortestPath_LR('TSP_instance_n_50_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 2548 rows, 122500 columns and 343637 nonzeros
Model fingerprint: 0xf30718d1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 101 rows and 9511 columns
Presolve time: 0.37s
Presolved: 2447 rows, 112989 columns, 338923 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.281e+05
 Factor NZ  : 2.909e+05 (roughly 50 MBytes of memory)
 Factor Ops : 3.581e+07 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.12320712e+05  1.32500461e+01  2.31e+02 1.28e+01  1.10e+00     1s

<gurobi.Model Continuous instance Shortest Path_copy: 2548 constrs, 122500 vars, Parameter changes: TimeLimit=600.0>

In [24]:
shortestPath('TSP_instance_n_50_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 2548 rows, 122500 columns and 343637 nonzeros
Model fingerprint: 0xc8819e64
Variable types: 0 continuous, 122500 integer (122500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 555.5928697
Presolve removed 96 rows and 9506 columns
Presolve time: 0.91s
Presolved: 2452 rows, 112994 columns, 336630 nonzeros
Variable types: 0 continuous, 112994 integer (112994 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    4613    4.5626411e+01   5.281869e+01   0.000000e+00      5s
    8742    6.6783752e+01   2.611405e+02   0.000000e+00     10s
   11939    8.2739551e+01   2.306874e+02   0.000000e+00     15s
   14870    8.9040533e+01   2

<gurobi.Model MIP instance Shortest Path: 2548 constrs, 122500 vars, Parameter changes: TimeLimit=600.0>

In [25]:
shortestPath_LR('TSP_instance_n_50_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 2548 rows, 122500 columns and 343637 nonzeros
Model fingerprint: 0x1fc1bb46
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 101 rows and 9511 columns
Presolve time: 0.38s
Presolved: 2447 rows, 112989 columns, 338923 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.281e+05
 Factor NZ  : 2.909e+05 (roughly 50 MBytes of memory)
 Factor Ops : 3.581e+07 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.20191324e+05  2.50776024e+00  2.31e+02 2.07e+00  7.77e-01     1s

<gurobi.Model Continuous instance Shortest Path_copy: 2548 constrs, 122500 vars, Parameter changes: TimeLimit=600.0>

The following are the Shortest Path formulation integer and Linear Relaxation solutions for two instances of size 70.

In [26]:
shortestPath('TSP_instance_n_70_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4968 rows, 338100 columns and 966897 nonzeros
Model fingerprint: 0xf4e323f5
Variable types: 0 continuous, 338100 integer (338100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 772.9329969
Presolve removed 136 rows and 18906 columns
Presolve time: 3.00s
Presolved: 4832 rows, 319194 columns, 952890 nonzeros
Variable types: 0 continuous, 319194 integer (319194 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    1516    2.6395490e+01   2.250000e+01   0.000000e+00      5s
    3478    3.7144214e+01   2.552055e+01   0.000000e+00     10s
    5442    4.4371259e+01   1.066528e+02   0.000000e+00     15s
    6893    4.9157251e+01  

<gurobi.Model MIP instance Shortest Path: 4968 constrs, 338100 vars, Parameter changes: TimeLimit=600.0>

In [27]:
shortestPath_LR('TSP_instance_n_70_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4968 rows, 338100 columns and 966897 nonzeros
Model fingerprint: 0x80d7f657
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 141 rows and 18911 columns
Presolve time: 1.01s
Presolved: 4827 rows, 319189 columns, 957503 nonzeros

Ordering time: 0.02s

Barrier statistics:
 AA' NZ     : 6.428e+05
 Factor NZ  : 8.146e+05 (roughly 140 MBytes of memory)
 Factor Ops : 1.415e+08 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.23410345e+05  1.32500461e+01  4.63e+02 1.28e+01  1.10e+00     

<gurobi.Model Continuous instance Shortest Path_copy: 4968 constrs, 338100 vars, Parameter changes: TimeLimit=600.0>

In [28]:
shortestPath('TSP_instance_n_70_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4968 rows, 338100 columns and 966897 nonzeros
Model fingerprint: 0x83235025
Variable types: 0 continuous, 338100 integer (338100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 709.9208865
Presolve removed 136 rows and 18906 columns
Presolve time: 2.89s
Presolved: 4832 rows, 319194 columns, 952890 nonzeros
Variable types: 0 continuous, 319194 integer (319194 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    1633    3.1749628e+01   2.912500e+01   0.000000e+00      5s
    3582    4.1970201e+01   2.946667e+01   0.000000e+00     10s
    5554    4.8436067e+01   9.363730e+01   0.000000e+00     15s
    7503    5.4137728e+01  

<gurobi.Model MIP instance Shortest Path: 4968 constrs, 338100 vars, Parameter changes: TimeLimit=600.0>

In [29]:
shortestPath_LR('TSP_instance_n_70_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4968 rows, 338100 columns and 966897 nonzeros
Model fingerprint: 0x5b3c7f14
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 141 rows and 18911 columns
Presolve time: 1.27s
Presolved: 4827 rows, 319189 columns, 957503 nonzeros

Ordering time: 0.02s

Barrier statistics:
 AA' NZ     : 6.428e+05
 Factor NZ  : 8.146e+05 (roughly 140 MBytes of memory)
 Factor Ops : 1.415e+08 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.39619605e+05  2.50776024e+00  4.63e+02 2.07e+00  7.88e-01     

<gurobi.Model Continuous instance Shortest Path_copy: 4968 constrs, 338100 vars, Parameter changes: TimeLimit=600.0>